In [1]:
import torch
from torch import nn

# 单隐藏层的MLP
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))

X = torch.rand(size=(2, 4))
net(X)

tensor([[0.2282],
        [0.3467]], grad_fn=<AddmmBackward0>)

In [2]:
# 参数访问
# args第三个是nn.Linear(8,1),看看长啥样
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.2538,  0.2928, -0.1281, -0.1414, -0.0675,  0.0224,  0.2787,  0.0976]])), ('bias', tensor([0.1657]))])


In [3]:
# 访问更具体的参数
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.1657], requires_grad=True)
tensor([0.1657])


In [4]:
# 可以直接访问
print(net[2].weight.grad)

None


In [5]:
# 一次性访问所有参数
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [6]:
net.state_dict()['2.bias'].data

tensor([0.1657])

In [7]:
# 从嵌套块收集参数
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())


def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())  # 可以为子模块命名
    return net


rg_net = nn.Sequential(block2(), nn.Linear(4, 1))
print(rg_net)
print(rg_net(X))

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)
tensor([[-0.0443],
        [-0.0443]], grad_fn=<AddmmBackward0>)


In [8]:
# 内置初始化
def init_normal(m):
    """初始化为均值为0，方差为0.01的正态分布"""
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)


net.apply(init_normal)  # 应用到每一层
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0223,  0.0042, -0.0043, -0.0002]), tensor(0.))

In [9]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)


net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

思考: Weight为什么不能初始化为常数?

对称性问题和梯度消失等问题

In [10]:
# 对某些块应用不同的初始化方法
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)


def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)


net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data[0])

tensor([-0.0408, -0.2486, -0.2578, -0.2764])
tensor([42., 42., 42., 42., 42., 42., 42., 42.])


In [11]:
# 参数绑定
shared = nn.Linear(8, 8)
# 第二层和第四层是同一个对象，所以它们共享参数 ——如何在不同的网络之间共享权重的方法
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared,
                    nn.ReLU(), nn.Linear(8, 1))
net(X)

tensor([[-0.2428],
        [-0.2381]], grad_fn=<AddmmBackward0>)

In [12]:
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
